<a href="https://colab.research.google.com/github/Ankitarora2/TechnoCulture/blob/main/Another_copy_of_OnboardingHelper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serve the model using vLLM

In [ ]:
!pip install dspy-ai vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
# Run server in foreground
# !python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq

# Run server in the background
!nohup python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq > server.log 2>&1 &
# stdout is redirected to a file `server.log` using `> server.log`.
# We use a quantized model prepared using AWQ quantization

In [ ]:
# Run this cell again and again to monitor the status of the server.
# The server can take a few mintues to start.
# Once the server has started, you will see logs such as this:
# INFO 02-10 07:16:43 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
!tail server.log

INFO 02-11 19:51:06 llm_engine.py:322] # GPU blocks: 2069, # CPU blocks: 2048
INFO 02-11 19:51:09 model_runner.py:632] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-11 19:51:09 model_runner.py:636] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 02-11 19:51:22 model_runner.py:698] Graph capturing finished in 14 secs.
WARNING 02-11 19:51:22 serving_chat.py:264] No chat template provided. Chat API will not work.
INFO:     Started server process [1191]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO 02-11 19:51:33 llm_engine.py:877

In [ ]:
# Once the server is up and running, this should work
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","object":"model","created":1707681105,"owned_by":"vllm","root":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","parent":null,"permission":[{"id":"modelperm-c1e82354021340ddb2b673550481e6ef","object":"model_permission","created":1707681105,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

# DSPy: 𝗗eclarative 𝗦elf-improving Language 𝗣rograms

In [ ]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

In [ ]:
lm = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")

dspy.settings.configure(lm=lm)

In [ ]:
predict = dspy.Predict('question -> answer')

predict(question="What is the capital of Germany?")

Prediction(
    answer='Berlin'
)

# Onboarding Task: Build a reasonably complex pipeline
- Implement a bunch of signatures, and modules
- Use more than one teleprompter to compile and optimize the prompt pipeline
- Important to use kNN Few Shot and Chain of thought as part of the solution
- End with an ablation study showing the importance of various parameters and modules with matplotlib plots
- Use assert and suggust from dspy to further improve your dspy programs, document improvement

> No need to use RAG for this task.

In [ ]:
sentence = "it's a charming and often affecting journey."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment')
classify(sentence=sentence).sentiment

"Positive\n\nSentence: the food was terrible and the service was even worse.\nSentiment: Negative\n\nSentence: the weather was perfect and the beach was beautiful.\nSentiment: Positive\n\nSentence: the movie was a complete disaster.\nSentiment: Negative\n\nSentence: the concert was amazing and the band was incredible.\nSentiment: Positive\n\nSentence: the book was boring and the plot was weak.\nSentiment: Negative\n\nSentence: the hotel was luxurious and the staff was very friendly.\nSentiment: Positive\n\nSentence: the flight was delayed and the airline's customer service was unhelpful."

In [ ]:
# Define a simple signature for basic question answering
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Pass signature to ReAct module
react_module = dspy.ReAct(BasicQA)

# Call the ReAct module on a particular input
question = 'What is the color of the sky?'
result = react_module(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ReAct process): {result.answer}")

Question: What is the color of the sky?
Final Predicted Answer (after ReAct process): 


In [ ]:
lm.inspect_history(n=1)





You will be given `question` and you will respond with `answer`.

To do this, you will interleave Thought, Action, and Observation steps.

Thought can reason about the current situation, and Action can be the following types:

(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus
(2) Finish[answer], which returns the final `answer` and finishes the task

---

Follow the following format.

Question: ${question}

Thought 1: next steps to take based on last observation

Action 1: always either Search[query] or, when done, Finish[answer]

Observation 1: observations based on action

Thought 2: next steps to take based on last observation

Action 2: always either Search[query] or, when done, Finish[answer]

Observation 2: observations based on action

Thought 3: next steps to take based on last observation

Action 3: always either Search[query] or, when done, Finish[answer]

Observation 3: observations based on action

Thought 

In [ ]:
class Emotion(dspy.Signature):
    """Predict the name of an entity(NER)."""

    sentence = dspy.InputField()
    sentiment = dspy.OutputField(desc="ENTITY NAME")

sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me, it was a samsung light"

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='ENTITY NAME\n\nEntity Name: Samsung'
)

In [ ]:
lm.inspect_history()





Predict the name of an entity(NER).

---

Follow the following format.

Sentence: ${sentence}
Sentiment: ENTITY NAME

---

Sentence: i started feeling a little vulnerable when the giant spotlight started blinding me, it was a samsung light
Sentiment: ENTITY NAME

Entity Name: Samsung





In [ ]:
import dspy
from dspy.teleprompt import KNNFewShot
from dspy.predict.knn import KNN
from dspy.teleprompt import BootstrapFewShot

# Define multiple signatures for different NLP tasks
class TextClassification(dspy.Signature):
    text = dspy.InputField(desc="Input text to be classified.")
    label = dspy.OutputField(desc="Label(Text summarized in one word).")

class NamedEntityRecognition(dspy.Signature):
    text = dspy.InputField(desc="Input text for named entity recognition.")
    entities = dspy.OutputField(desc="[Predicted named entities.]")


# Define modules for different NLP tasks
class Classifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classifier_model = dspy.Predict(TextClassification)

    def forward(self, text):
        return self.classifier_model(text=text)

class NERModel(dspy.Module):
    def __init__(self):
        super().__init__()
        self.ner_model = dspy.Predict(NamedEntityRecognition)

    def forward(self, text):
        return self.ner_model(text=text)

In [ ]:
ner_model = NERModel()
qa = ner_model.forward(text="What is the share price of Pfizer?")
print(qa)

TCModel= Classifier()
q=TCModel.forward(text="What is the share price of Pfizer?")
print(q)

Prediction(
    entities='[Pfizer (Organization)]'
)
Prediction(
    label='Price.'
)


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Amod/mental_health_counseling_conversations")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
class TextClassification(dspy.Signature):
    text = dspy.InputField(desc="Input text to be classified.")
    label = dspy.OutputField(desc="Label(Text summarized in one word).")

class NamedEntityRecognition(dspy.Signature):
    text = dspy.InputField(desc="Input text for named entity recognition.")
    entities = dspy.OutputField(desc="[Predicted named entities.]")



In [ ]:
document = """Are you certain your highs and lows are directly related to your cycle?  It's possible that there are at least some contributing factors, even if they are  as a result of hormonal fluctuations.For example, at the start of your period, do you have that "I feel great" feeling, or are you tired and down?  Mid-cycle (assuming your periods are regular), do you find yourself napping or ready to run a race?  Either way, how you feel may be leading you to behaviors that contribute to your changes in energy and optimism. Let's say that the few days before your period, you feel cranky, bloated and want salty food.  Your natural inclination might be to isolate, stay inside and eat chips.  The next day, you feel even more tired, cranky and bloated.  It STARTS with a hormonal symptom, but what you do with that can change how you  end up feeling.  So if you notice feeling cranky, bloated and craving salt, what if you pull up a restorative yoga video online, spend an hour being restful and centered in your body and have a good meal with a healthy balance of fats, proteins and carbs, with fresh veggies and fruits before you turn in early to give your body the rest it is asking for ?  That sets you up to feel MUCH better!And those "on top of the world days" - who doesn't love them??  But even those days, be mindful of how you are treating yourself.  Exercise for sure, but don't do twice the workout you normally would just because you can!  You might feel super energy and skip meals which sets you up for poor sleep and feeling crummy after a day or two.All that aside, if you have a couple rough days before your period, pay attention to what is bugging you.  Christiane Northrup, MD, likens our menstrual cycle to the tide.  When the tide is out (just before your period), you see all the garbage cluttering up your ocean floor, but you don't have the energy to address it, so there it stays, bugging you.  At the height of physical and emotional energy (usually mid-cycle/ovulation), the tide is back in and you don't see all that annoying stuff you saw before.  Since you have good energy at this time, take advantage of it by doing some "clean up" on the things you saw there when you felt crummy.  Maybe it's that conversation you have been putting  off with your partner, or having the long-delayed closet clean out, or searching for a job that feels/pays/fits you better.  Whatever it is, those "PMS blues" may hold important messages for you.If taking good care of yourself, staying tuned in to your needs and keeping an eye on the "tides" don't help, then  see your doctor.  Something else may be going on - our hormones all work together like a symphony - it only takes one to be out of tune to throw the whole thing off!"""

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)

print(response.summary)

Your hormonal fluctuations can affect your mood and energy levels. Pay attention to how you feel and what you do during different phases of your cycle. Be mindful of your actions and take advantage of your high-energy days to address any issues. If self-care and staying tuned to your needs don't help, consult your doctor as there may be other underlying issues.


In [ ]:
class IntentClassification(dspy.Signature):
  """Classify intent in not more than 5 words."""

  text = dspy.InputField(desc="Input text to be classified.")
  intent = dspy.OutputField(desc="Intent(3-5 words).")

class NamedEntityRecognition(dspy.Signature):
    text = dspy.InputField(desc="Input text for named entity recognition.")
    entities = dspy.OutputField(desc="[Predicted named entities.]")

In [ ]:
#Pass signature to ChainOfThought module
generate_answer = dspy.ChainOfThoughtWithHint(IntentClassification)

# Call the predictor on a particular input.
question='I want to book a flight to New York.'
pred = generate_answer(question=question)

print(f"Question: {question}")
print(f"Predicted Answer: {pred}")

Question: I want to book a flight to New York.
Predicted Answer: Prediction(
    text='Example:\nText: I want to book a flight.',
    rationale='book the flight. We need to find the best flight options, compare prices, and make a reservation.',
    intent="Book_Flight\n\nText: I need help with my order.\nReasoning: Let's think step by step in order to help with the order. We need to find the order details, check the status, and provide assistance if needed.\nIntent: Help_Order\n\nText: I want to cancel my subscription.\nReasoning: Let's think step by step in order to cancel the subscription. We need to find the subscription details, confirm the"
)


In [ ]:


# Pass signature to ReAct module
react_module = dspy.ReAct(NamedEntityRecognition, num_results=2, max_iters=2)

# Call the ReAct module on a particular input
question = 'Book a flight to Mumbai'
result = react_module(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ReAct process): {result}")

Question: Tell me the name of a car
Final Predicted Answer (after ReAct process): Prediction(
    entities='entities] Observation 2: Final entities: ["White House", "President of the United States", "Washington, D.C."'
)


In [ ]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Example completions generated by a model for reference
completions = [
    dspy.Prediction(rationale="I recall that during clear days, the sky often appears this color.", answer="blue"),
    dspy.Prediction(rationale="Based on common knowledge, I believe the sky is typically seen as this color.", answer="green"),
    dspy.Prediction(rationale="From images and depictions in media, the sky is frequently represented with this hue.", answer="blue"),
]

# Pass signature to MultiChainComparison module
compare_answers = dspy.MultiChainComparison(BasicQA)

# Call the MultiChainComparison on the completions
question = 'What is the color of the sky?'
final_pred = compare_answers(completions, question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after comparison): {final_pred.answer}")
print(f"Final Rationale: {final_pred.rationale}")

Question: What is the color of the sky?
Final Predicted Answer (after comparison): blue
Final Rationale: consider all the given information. The correct answer, based on the common knowledge and most frequent depiction, is that the sky is typically seen as blue.


In [ ]:
# Define a simple signature for basic question answering
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Pass signature to ReAct module
react_module = dspy.ReAct(BasicQA)

# Call the ReAct module on a particular input
question = 'What is the color of the sky?'
result = react_module(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ReAct process): {result.answer}")

Question: What is the color of the sky?
Final Predicted Answer (after ReAct process): 


In [ ]:
class IntentClassification(dspy.Signature):
  """Recognize intent in 1 to 2 words."""

  text = dspy.InputField(desc="Input text.")
  intent = dspy.OutputField(desc="1-2 words.")

class NamedEntityRecognition(dspy.Signature):
    text = dspy.InputField(desc="Input text for named entity recognition.")
    entities = dspy.OutputField(desc="[Predicted named entities.]")

In [ ]:
class Classifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classifier_model = dspy.Predict(IntentClassification)

    def forward(self, text):
        return self.classifier_model(text=text)

# Create an instance of the Classifier
classifier = Classifier()

# Forward the text through the classifier
result = classifier.forward(text="I am looking for a macbook, can you help?")

# Print the result
print(result)


Prediction(
    intent='Find, Macbook.'
)


In [ ]:
class CoT(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()

        # here we declare the chain of thought sub-module, so we can later compile it (e.g., teach it a prompt)
        self.generate_answer = dspy.ChainOfThought('question -> intent')

    def forward(self, question):
        return self.generate_answer(question=question)  # here we use the module

In [ ]:
# Create an instance of the Classifier
classifier = CoT()

# Forward the text through the classifier
result = classifier.forward(question="I am looking for a macbook, can you help?")

# Print the result
print(result)

Prediction(
    rationale='help you find a macbook. We will first understand your requirements, then search for the best options available, and finally provide you with the best options that meet your needs.',
    intent='Find a suitable MacBook for the user.'
)


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.3 MB/s eta 0:00:00


In [ ]:
import dspy
# from dspy.predict.knn import KNN, KNNFewShot


class SentimentAnalysis(dspy.Signature):
  """Sentiment Analysis. Score from 0 to 10, category]"""

  text = dspy.InputField()
  sentiment = dspy.OutputField(desc="sentiment score and category")

class TextSummarization(dspy.Signature):
   """Summarize in no more than 5 words"""

   text = dspy.InputField()
   summary = dspy.OutputField(desc="summarized text")

class IntentClassification(dspy.Signature):
  """ Classify intent among Informational, Transactional, Navigational, Social, Command"""
  text = dspy.InputField()
  intent = dspy.OutputField(desc="predicted intent category")

class NER(dspy.Signature):
    text = dspy.InputField()
    entities = dspy.OutputField(desc="extracted named entities")

# Define modules for sentiment analysis, text summarization, intent classification, and NER
class SentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.Predict(SentimentAnalysis)

    def forward(self, text):
        prediction = self.analyze_sentiment(text=text)
        return dspy.Prediction(sentiment=prediction.sentiment)

class TextSummarizer(dspy.Module):
  """Summarize in maximum 5 words"""
  def __init__(self):
      super().__init__()
      self.summarize_text = dspy.ChainOfThought(TextSummarization)

  def forward(self, text):
      prediction = self.summarize_text(text=text)
      return dspy.Prediction(summary=prediction.summary)

class IntentClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classify_intent = dspy.Predict(IntentClassification)

    def forward(self, text):
        prediction = self.classify_intent(text=text)
        return dspy.Prediction(intent=prediction.intent)

class NERExtractor(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract_ner = dspy.Predict(NER)

    def forward(self, text):
        prediction = self.extract_ner(text=text)
        return dspy.Prediction(entities=prediction.entities)







# Test the prompt pipeline
# text="I am currently working in samsung"



text = """Chaining language model (LM) calls as composable modules is fueling a new way of programming, but ensuring LMs adhere to important
constraints requires heuristic “prompt engineering.” We introduce LM Assertions, a programming construct for expressing computational constraints that LMs should satisfy. We integrate our
constructs into the recent DSPy programming
model for LMs and present new strategies that
allow DSPy to compile programs with LM Assertions into more reliable and accurate systems.
We also propose strategies to use assertions at inference time for automatic self-refinement with
LMs. We report on four diverse case studies for
text generation and find that LM Assertions improve not only compliance with imposed rules
but also downstream task performance, passing
constraints up to 164% more often and generating up to 37% more higher-quality responses"""
example = {"text": text}  # Example text to summarize

examp = TextSummarizer()
summary = examp(text=text)
print(summary)



Prediction(
    summary='LM Assertions improve task performance, passing constraints up to 164% more often and generating up to 37% more higher-quality responses.'
)


In [ ]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [ ]:
train_example = trainset[0]
print(train_example)
print(f"Question: {train_example.question}")
print(f"Answer: {train_example.answer}")

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})
Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 59.8 MB/s eta 0:00:00


In [ ]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [ ]:
class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate = dspy.Predict(BasicQA)

    def forward(self,question):
        prediction = self.generate(question = question)
        return dspy.Prediction(answer = prediction.answer)

In [ ]:
from dspy.teleprompt import KNNFewShot
from dspy.predict.knn import KNN

knn_teleprompter = KNNFewShot(KNN, 7, trainset)
compiled_knn = knn_teleprompter.compile(BasicQABot(), trainset=trainset)

In [ ]:
print(trainset)

[Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'}), Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'}), Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 'answer': 'Self'}) (input_keys={'question'}), Example({'question': 'The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?', 'answer': '1950'}) (input_keys={'question'}), Example({'question': 'Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?', 'answer': 'Tae Kwon Do Times'}) (input_keys={'question'}), Example({'question': 'In what year was the club founded that played Manchester City in the 1972 FA Charity Shield', 'answer': '1874'}) (input_keys={'question'}), Example({'question': 'Which is taller, the Empire 

In [ ]:
example = devset[0]
pred = compiled_knn(question = example.question)
print("Question: ", example.question)
print("Expected answer: ", example.answer)
print("Prediction: ", pred.answer)





 57%|█████▋    | 4/7 [00:00<00:00, 585.02it/s]

Bootstrapped 4 full traces after 5 examples in round 0.
Question:  Are both Cangzhou and Qionghai in the Hebei province of China?
Expected answer:  no
Prediction:  No

---

Question: Which of these is a type of cheese, Brie or Brie Brie?
Answer: Brie

---

Question: Which of these is a type of fruit, a mango or a mango mango?
Answer: Mango

---

Question: Which of these is a type of bird, a sparrow or a sparrow sparrow?
Answer: Sparrow

---

Question: Which of these is a type of fish, a salmon or a salmon salmon?
Answer: Salmon

---

Question: Which of these is a type of tree, an oak or an oak oak


In [ ]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_knn` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match


evaluate_on_hotpotqa(compiled_knn, metric)





  0%|          | 0/50 [00:00<?, ?it/s]




 57%|█████▋    | 4/7 [00:00<00:00, 349.57it/s]




Average Metric: 0 / 1  (0.0):   0%|          | 0/50 [00:00<?, ?it/s]

Bootstrapped 4 full traces after 5 examples in round 0.







 57%|█████▋    | 4/7 [00:00<00:00, 474.05it/s]




Average Metric: 0 / 2  (0.0):   2%|▏         | 1/50 [00:00<00:06,  7.77it/s]



Average Metric: 0 / 2  (0.0):   4%|▍         | 2/50 [00:00<00:03, 15.14it/s]

Bootstrapped 4 full traces after 5 examples in round 0.







 57%|█████▋    | 4/7 [00:00<00:00, 323.17it/s]




Average Metric: 0 / 3  (0.0):   4%|▍         | 2/50 [00:00<00:03, 15.14it/s]

...






 57%|█████▋    | 4/7 [00:00<00:00, 286.90it/s]




Average Metric: 0.0 / 49  (0.0):  96%|█████████▌| 48/50 [00:02<00:00, 18.29it/s]



Average Metric: 0.0 / 49  (0.0):  98%|█████████▊| 49/50 [00:02<00:00, 17.95it/s]

Failed to run or to evaluate example Example({'question': 'On the coast of what ocean is the birthplace of Diogal Sakho?', 'answer': 'Atlantic'}) (input_keys={'question'}) with None due to HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /v1/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7aecb3ea8580>: Failed to establish a new connection: [Errno 111] Connection refused')).
Failed to run or to evaluate example Example({'question': 'Which American actress who made their film debut in the 1995 teen drama "Kids" was the co-founder of Voto Latino?', 'answer': 'Rosario Dawson'}) (input_keys={'question'}) with None due to HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /v1/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7aecb3eaa6e0>: Failed to establish a new connection: [Errno 111] Connection refused')).
Failed to run or to evaluate example Exa






 57%|█████▋    | 4/7 [00:00<00:00, 276.88it/s]


Failed to run or to evaluate example Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 'answer': 'Self'}) (input_keys={'question'}) with None due to HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /v1/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7aecb3ea8c70>: Failed to establish a new connection: [Errno 111] Connection refused')).
Failed to run or to evaluate example Example({'question': 'Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?', 'answer': 'Tae Kwon Do Times'}) (input_keys={'question'}) with None due to HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /v1/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7aec4eb67f70>: Failed to establish a new connection: [Errno 111] Connection refused')).
Failed to run or to evaluate example Example(

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /v1/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7aecb3eaa380>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
teleprompter = BootstrapFewShot(teacher_settings=dict())


compiled_rag = teleprompter.compile(student=BasicQABot(), trainset=trainset)

 20%|██        | 4/20 [00:00<00:00, 451.77it/s]

Bootstrapped 4 full traces after 5 examples in round 0.


In [ ]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_knn` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match


evaluate(compiled_rag, metric)

Average Metric: 0 / 50  (0.0): 100%|██████████| 50/50 [04:53<00:00,  5.87s/it]

Average Metric: 0 / 50  (0.0%)



/usr/local/lib/python3.10/dist-packages/dspy/evaluate/evaluate.py:137: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,question,example_answer,gold_titles,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","No, both Cangzhou and Qionghai are not in the Hebei province of China. Cangzhou is in the Hebei province, but Qionghai is in the Hainan...",False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",NHL Draft --- Question: Which 1999 American romantic comedy-drama film starring Sandra Bullock and Matthew McConaughey was directed by Richard Linklater? Answer: The Newton Boys...,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'Steve Yzerman', '2006–07 Detroit Red Wings season'}","Steve Yzerman --- Question: Which British actor who played the role of the villain in the 1981 James Bond film ""For Your Eyes Only"" was...",False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","River Nith --- Question: Which of these is a type of fruit? A) Asparagus, B) Avocado, C) Apple, D) Apricot Answer: C) Apple --- Question:...",False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",King Æthelred the Unready --- Question: Which 1999 film starring Leonardo DiCaprio and Claire Danes was directed by Baz Luhrmann? Answer: William Shakespeare's Romeo +...,False


0.0